In [13]:
import openmm
import urllib

In [39]:
from openmm import app, unit
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/openmm/openmm/master/wrappers/python/tests/systems/alanine-dipeptide-explicit.pdb",
    "input.pdb"
)
pdb = app.PDBFile('input.pdb')

In [40]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

In [41]:
system = forcefield.createSystem(
    pdb.topology, 
    nonbondedMethod=app.PME,
    nonbondedCutoff=1*unit.nanometer, 
    constraints=app.HBonds,
)

In [42]:
integrator = openmm.LangevinIntegrator(
    300*unit.kelvin, 
    1.0/unit.picoseconds, 
    2.0*unit.femtoseconds
)

In [43]:
simulation = app.Simulation(pdb.topology, system, integrator)

In [44]:
simulation.context.setPositions(pdb.positions)

In [45]:
simulation.minimizeEnergy()

In [46]:
simulation.reporters.append(app.PDBReporter('output.pdb', 10))

In [47]:
simulation.step(100)

In [48]:
import mdtraj as md
traj = md.load('output.pdb')

In [49]:
import nglview
view = nglview.show_mdtraj(traj)

In [50]:
md.compute_J3_HN_C(traj);
md.density(traj);